# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : High dimensional problems
## Chapter : Minimal path for a rolling ball

In this notebook, we present a numerical solution to the rolling ball problem: find the optimal way to roll a ball on a surface, so that it reaches a given position and orientation. The main properties of this problem are as follows:
* *High dimensional.* The configuration space of the system is 5 dimensional: three dimensions for the orientation of the ball, and two dimensions for its position on the surface. Arguably, 5 dimensions is (slightly past) the threshold where discretizations on uniform grids loose their relevance.
* *Strongly anisotropic* The problem is of sub-Riemannian type, and its anisotropy is fully generic, not aligned with the axes of the discretization grid.

The rolling ball problem admits numerous variants. One may for instance roll a ball along a one dimensional line, or on a more complex surface such as another ball.

## 0. Importing the required libraries

In [5]:
import sys; sys.path.insert(0,"../..")

In [278]:
from agd import AutomaticDifferentiation as ad
from agd import LinearParallel as lp
from agd import Metrics
norm = ad.Optimization.norm

In [265]:
import numpy as np; xp=np
import scipy.linalg
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%5.3g" % x))

## 1. Description of the models

### 1.1 Conversions

**Rotations and unit quaternions**

In [1]:
def rotation_from_quaternion(q):
    """Produces the rotation associated with a unit quaternion."""
    qr,qi,qj,qk = q
    return xp.eye(3) + 2*ad.array([
        [-(qj**2+qk**2), qi*qj-qk*qr, qi*qk+qj*qr],
        [qi*qj+qk*qr, -(qi**2+qk**2), qj*qk-qi*qr],
        [qi*qk-qj*qr, qj*qk+qi*qr, -(qi**2+qj**2)]])
def quaternion_from_rotation(r):
    """Produces the unit quaternion, with positive real part, associated with a rotation."""
    qr = np.sqrt(lp.trace(r)+1.)/2.
    qi = (r[2,1]-r[1,2])/(4*qr)
    qj = (r[0,2]-r[2,0])/(4*qr)
    qk = (r[1,0]-r[0,1])/(4*qr)
    return ad.array([qr,qi,qj,qk])

**Euclidean space and quaternions**

In [116]:
def quaternion_from_euclidean(e):
    """Produces a point in the unit sphere by projecting a point in the equator plane."""
    e2 = lp.dot_VV(e,e)
    return ad.array([1.-e2,*(2*e)])/(1.+e2)
def euclidean_from_quaternion(q):
    """Produces a point in the equator plane from a point in the unit sphere."""
    e2 = (1-q[0])/(1+q[0])
    return q[1:]*((1+e2)/2.)

In [117]:
def euclidean_from_rotation(r): return euclidean_from_quaternion(quaternion_from_rotation(r))
def rotation_from_euclidean(e): return rotation_from_quaternion(quaternion_from_euclidean(e))

In [118]:
def check(e):
    q  = quaternion_from_euclidean(e)
    r  = rotation_from_quaternion(q)
    q_ = quaternion_from_rotation(r)
    e_ = euclidean_from_quaternion(q_)
    assert np.allclose( norm(q,axis=0), 1.)
    assert np.allclose(lp.dot_AA(lp.transpose(r),r),np.eye(3))
    if norm(e,axis=0)<1: 
        assert np.allclose(q,q_)
        assert np.allclose(e,e_)

In [119]:
check(np.array([1.,2.,3.]))

**Antisymmetric matrices and their exponentials**

In [120]:
def antisym(a,b,c):
    return ad.array([[0, -c, b], [c, 0, -a], [-b, a, 0]])
def exp_antisym(a,b,c):
    """Matrix exponential of antisym(a,b,c)"""
    s = ad.asarray(a**2+b**2+c**2)
    s[s==0]=1e-20 # Same trick as in numpy's sinc function ...
    sq = np.sqrt(s)
    co,si = np.cos(sq),np.sin(sq)
    cosc,sinc = (1-co)/s,si/sq    
    return ad.array([
        [co+cosc*a**2, cosc*a*b-sinc*c, cosc*a*c+sinc*b],
        [cosc*a*b+sinc*c, co+cosc*b**2, cosc*b*c-sinc*a],
        [cosc*a*c-sinc*b, cosc*b*c+sinc*a, co+cosc*c**2]])

In [121]:
assert np.allclose(exp_antisym(1,2,3),scipy.linalg.expm(antisym(1,2,3)))

In [122]:
exp_antisym(1,2,3),scipy.linalg.expm(antisym(1,2,3))

(array([[-0.69492056,  0.71352099,  0.08929286],
        [-0.19200697, -0.30378504,  0.93319235],
        [ 0.69297817,  0.6313497 ,  0.34810748]]),
 array([[-0.69492056,  0.71352099,  0.08929286],
        [-0.19200697, -0.30378504,  0.93319235],
        [ 0.69297817,  0.6313497 ,  0.34810748]]))

### 1.2 Action of a control on a state

In this notebook, controls are antisymmetric matrices described by their coordinates in $R^3$. They act additively on the physical state, and multiplicatively (through their exponential) on the angular state.

In [183]:
def advance(state,control):
    state,control = map(ad.asarray,(state,control))
    state_physical = state[:-3]
    state_physical = state_physical + 0.25*control[:len(state_physical)] # Additive action on the physical state
    
    state_angular = rotation_from_euclidean(state[-3:])
    state_angular = lp.dot_AA(state_angular,exp_antisym(*control)) # Left invariant action
    
    return np.concatenate([state_physical,euclidean_from_rotation(state_angular)],axis=0)

Close to the origin, the three controls act independently on each coordinate of the euclidean parametrization of the sphere.

In [184]:
δ = ad.Dense.identity(constant=0.)
advance([0,0,0],[δ,0,0])

denseAD(array([0., 0., 0.]),
array([[0.25],
       [0.  ],
       [0.  ]]))

In [185]:
advance([0,0,0],[0,δ,0])

denseAD(array([0., 0., 0.]),
array([[0.  ],
       [0.25],
       [0.  ]]))

In [186]:
advance([0,0,0],[0,0,δ])

denseAD(array([0., 0., 0.]),
array([[0.  ],
       [0.  ],
       [0.25]]))

They also act independently on the plane or line parametrization, if present.

In [187]:
advance([0,0,0,0,0],[δ,0,0])

denseAD(array([0., 0., 0., 0., 0.]),
array([[0.25],
       [0.  ],
       [0.25],
       [0.  ],
       [0.  ]]))

In [188]:
advance([0,0,0,0,0],[0,δ,0])

denseAD(array([0., 0., 0., 0., 0.]),
array([[0.  ],
       [0.25],
       [0.  ],
       [0.25],
       [0.  ]]))

More complex motions, known as commutators, are needed to control the sphere and the plane independently, or to control the sphere using only two of the three availabel controls.

### 1.3 Commutators

In [189]:
def commute(state,control1,control2):
    state,control1,control2 = map(ad.asarray,(state,control1,control2))
    state = advance(state, control1)
    state = advance(state, control2)
    state = advance(state,-control1)
    state = advance(state,-control2)
    return state 

On the sphere, the commutator of the first two controls is the third control. This allows to bring the sphere from any state to any other state using only two controls, albeit rather inefficiently since the motion along the third control generated by a commutator is of second order.

In [190]:
δ = ad.Dense2.identity(constant=0.)
commute([0,0,0],[δ,0,0],[0,δ,0]) # Motion generated by a commutator of the first two controls

denseAD2(array([0., 0., 0.]),
array([[0.],
       [0.],
       [0.]]),
array([[[0. ]],

       [[0. ]],

       [[0.5]]]))

In [198]:
advance([0,0,0],[0,0,δ**2]) # Same motion generated by the third control, at second order.

denseAD2(array([0., 0., 0.]),
array([[0.],
       [0.],
       [0.]]),
array([[[0. ]],

       [[0. ]],

       [[0.5]]]))

Likewise, on the sphere, the commutator of the second and third control is minus the first one.

In [194]:
commute([0,0,0],[0,δ,0],[0,0,δ]) # Motion generated by a commutator of the second and third controls

denseAD2(array([0., 0., 0.]),
array([[0.],
       [0.],
       [0.]]),
array([[[0.5]],

       [[0. ]],

       [[0. ]]]))

However, on the plane, which is a vector space, all controls commute. This observation allows to control the motion of the sphere without changing its position on the plane.

In [195]:
commute([0,0,0,0,0],[0,δ,0],[0,0,δ]) # Motion generated by a commutator of the second and third controls.
# Only the sphere is affected

denseAD2(array([0., 0., 0., 0., 0.]),
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]]),
array([[[0. ]],

       [[0. ]],

       [[0.5]],

       [[0. ]],

       [[0. ]]]))

In [197]:
advance([0,0,0,0,0],[δ**2,0,0]) # Motion generated by the first control, at second order. 
# Both the sphere and plane are affected

denseAD2(array([0., 0., 0., 0., 0.]),
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]]),
array([[[0.5]],

       [[0. ]],

       [[0.5]],

       [[0. ]],

       [[0. ]]]))

The rank of a sub-Riemannian model is the depth of the control commutators needed to generate the entire tangent space.
A rank-0 model is amounts to Riemannian manifold. The numerical simulation, using PDE techniques, of models of rank larger than or equal to two is novel to the knowledge of the author.

### 1.3 Ball models

We present the hamiltonians of the different models, in the form of the symmetric semi-definite matrix defining the dual metric, which makes sense since the models are sub-Riemannian. A more classical presentation, involving the position and impulsion variables, is presented in next section.

In [257]:
def hamiltonian(model):
    grad = model.gradient()
    return lp.dot_AA(lp.transpose(grad),grad)

In [258]:
δ = 4*ad.Dense.identity(constant=[0.,0.,0.]) # Three independent symbolic perturbations
ball_state = np.array([1,2,3])/4 # Some arbitrary state

The **Riemannian ball** is governed by the three controls, with the same strength on each axis, and its state space is the ball alone. It is thus a Riemannian model. At the origin, the Hamiltonian is the identity matrix.

In [239]:
hamiltonian(advance([0,0,0],δ))

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

At any point in space, this hamiltonian is a positive definite matrix, whose inverse is the Riemannian metric. Because of our choice of coordinates, this metric is *conformal*, in other words proportional to the euclidean metric (up to machine precision).

In [242]:
hamiltonian(advance(ball_state,delta))

array([[3.51562500e+00, 5.79425397e-13, 8.59534666e-13],
       [5.79425397e-13, 3.51562500e+00, 1.72795112e-12],
       [8.59534666e-13, 1.72795112e-12, 3.51562500e+00]])

The **Reeds shepp ball** is governed only two of the three controls, missing the middle one, and is a sub-Riemannian model of rank one. This model can also be regarded as describing a wheelchair-like vehicle moving on a ball.

In [252]:
δRS = [δ[0],0,δ[2]] # The y-axis control is lacking
hamiltonian(advance([0,0,0],δRS))

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 1.]])

At any point in space, this hamiltonian is a positive semi-definite matrix, with two non-zero eigenvalues.

In [253]:
ham = hamiltonian(advance(state,δRS))
ham, np.linalg.eigvalsh(ham)

(array([[ 1.953125,  0.78125 ,  1.5625  ],
        [ 0.78125 ,  3.125   , -0.78125 ],
        [ 1.5625  , -0.78125 ,  1.953125]]),
 array([2.63677881e-16, 3.51562500e+00, 3.51562500e+00]))

The **torqueless ball** is governed by two of the three controls, missing the last one. On the ball alone, this model is identical to the Reeds-Shepp model. However the situation changes when put on a plane, as discussed below.

In [254]:
δTL = [δ[0],δ[1],0]
hamiltonian(advance([0,0,0],δTL))

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])

In [255]:
ham = hamiltonian(advance(ball_state,δTL))
ham, np.linalg.eigvalsh(ham)

(array([[ 1.625   , -0.34375 , -1.71875 ],
        [-0.34375 ,  3.453125, -0.3125  ],
        [-1.71875 , -0.3125  ,  1.953125]]),
 array([-1.11022302e-16,  3.51562500e+00,  3.51562500e+00]))

### 1.4 Ball with support

The **Riemannian ball on a plane** is a rank one sub-Riemannian model. The ball may be rolled along both the $x$-axis and $y$-axis, and its rotation around the $z$-axis is also controlled.

In [273]:
plane_state = np.concatenate((np.zeros(2),ball_state)) # Some arbitrary plane-ball state
line_state  = np.concatenate((np.zeros(1),ball_state)) # Some arbitrary plane-ball state

The hamiltonian is degenerate, as expected, with two vanishing eigenvalues due to the fact that motion in the plane is tied to the rotation of the ball.

In [268]:
ham = hamiltonian(advance(np.zeros(5),δ))
ham,np.linalg.eigvalsh(ham)

(array([[    1,     0,     1,     0,     0],
        [    0,     1,     0,     1,     0],
        [    1,     0,     1,     0,     0],
        [    0,     1,     0,     1,     0],
        [    0,     0,     0,     0,     1]]),
 array([    0,     0,     1,     2,     2]))

The situation is the same at any point in space.

In [269]:
ham = hamiltonian(advance(plane_state,δ))
ham,np.linalg.eigvalsh(ham)

(array([[    1,     0,  0.25,  1.75, -0.625],
        [    0,     1, -1.25, 0.625,  1.25],
        [ 0.25, -1.25,  3.52, 5.79e-13, 8.6e-13],
        [ 1.75, 0.625, 5.79e-13,  3.52, 1.73e-12],
        [-0.625,  1.25, 8.6e-13, 1.73e-12,  3.52]]),
 array([-8e-17, 7.45e-17,  3.52,  4.52,  4.52]))

The **Reeds-Shepp ball on a line** is a rank two sub-Riemannian model. The ball may only be rolled along the $x$-axis, and rotated around the $z$-axis.
In order to generate a rotation around the $y$-axis, a commutator of these two motions is required. In order to generate a rotation around the $x$-axis, without any motion in the plane, a second order commutator is needed involving rotation around the $y$-axis and a rotation around the $z$-axis.

In [272]:
ham = hamiltonian(advance(np.zeros(4),δRS))
ham,np.linalg.eigvalsh(ham)

(array([[    1,     1,     0,     0],
        [    1,     1,     0,     0],
        [    0,     0,     0,     0],
        [    0,     0,     0,     1]]),
 array([    0,     0,     1,     2]))

In [275]:
ham = hamiltonian(advance(line_state,δRS))
ham,np.linalg.eigvalsh(ham)

(array([[    1,  0.25,  1.75, -0.625],
        [ 0.25,  1.95, 0.781,  1.56],
        [ 1.75, 0.781,  3.13, -0.781],
        [-0.625,  1.56, -0.781,  1.95]]),
 array([-3.37e-16, 4.61e-16,  3.52,  4.52]))

The **torqueless ball on a plane** is a rank two sub-Riemannian model. The ball may be rolled along the $x$-axis and the $y$-axis, but not rotated around the $z$-axis (think of a punctual control, without any torque). In order to generate a rotation around the $z$-axis, a commutator of the rolling motions along the $x$-axis and the $y$-axis is needed. In order to generate a pure rotation around the $x$-axis or the $y$-axis, one needs a commutator of the pure rotation around the $z$-axis, and of the rolling motion along the $x$ or the $y$ axis.

In [276]:
ham = hamiltonian(advance(np.zeros(5),δTL))
ham,np.linalg.eigvalsh(ham)

(array([[    1,     0,     1,     0,     0],
        [    0,     1,     0,     1,     0],
        [    1,     0,     1,     0,     0],
        [    0,     1,     0,     1,     0],
        [    0,     0,     0,     0,     0]]),
 array([    0,     0,     0,     2,     2]))

In [277]:
ham = hamiltonian(advance(plane_state,δTL))
ham,np.linalg.eigvalsh(ham)

(array([[    1,     0,  0.25,  1.75, -0.625],
        [    0,     1, -1.25, 0.625,  1.25],
        [ 0.25, -1.25,  1.62, -0.344, -1.72],
        [ 1.75, 0.625, -0.344,  3.45, -0.312],
        [-0.625,  1.25, -1.72, -0.312,  1.95]]),
 array([-3.67e-16, 7.34e-17, 5.62e-16,  4.52,  4.52]))

## 2. Geodesic shooting

We compute, for checking their consistency, some geodesics of the considered models model. They correspond to the free motion of the corresponding physical system, without external force or obstacles.

In [ ]:
def hamiltonian(controls):
    pass

In [279]:
help(Metrics.Hamiltonian)

Help on class Hamiltonian in module agd.Metrics.hamiltonian:

class Hamiltonian(builtins.object)
 |  Hamiltonian(H, shape_free=None, vdim=-1, disassociate_ad=False, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  DpH(self, q, p)
 |      Differentiates the Hamiltonian, w.r.t. impulsion.
 |  
 |  DqH(self, q, p)
 |      Differentiates the Hamiltonian, w.r.t. position.
 |  
 |  H(self, q, p)
 |      Evaluates the Hamiltonian, for a given position and impulsion.
 |  
 |  __init__(self, H, shape_free=None, vdim=-1, disassociate_ad=False, **kwargs)
 |      Inputs:
 |      - H : the hamiltonian, which may be either:
 |              * a metric
 |              * a callable function
 |              * a pair of callable functions, for a separable hamiltonian.
 |                      (in that case, may also be scalars or matrices, for quadratic hamiltonians)
 |      - shape_free (optional) : the shape of the position and impulsion
 |      - vdim (optional): equivalent to shape_free = (vdim,)
 | 

In [ ]:
ham